##Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [40]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False

###Préparation des matrices de traits

In [41]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')
#fs.supremum.concept.extent

In [42]:
bdlexiqueIn = unicode(u"èò")
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO")
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [43]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    return result

#Préparation du tableau de VERBES

###Lecture du lexique

In [44]:
nomLexique="/Users/gilles/Copy/Python/phonemisation/bdlexique-PDM.txt"
bdlexique=pd.read_csv(nomLexique,sep=";",names=["ortho","phono","ext","cs","ms","vs","lexeme","L23","lemmeFrantext","lemmeFilms","formeFrantext","formeFilms"],encoding="utf8")

###Définition des cases
- principales pour le paradigme courant
- secondaires pour les cases rares
- totales pour le tout

In [45]:
casesPrincipales= [
        'inf', 'pi1S', 'pi2S', 'pi3S', 'pi1P', 'pi2P', 'pi3P', 'ii1S',
        'ii2S', 'ii3S', 'ii1P', 'ii2P', 'ii3P', 
        'fi1S', 'fi2S', 'fi3S', 'fi1P', 'fi2P',
        'fi3P', 'pI2S', 'pI1P', 'pI2P', 'ps1S', 'ps2S', 'ps3S', 'ps1P',
        'ps2P', 'ps3P', 
        'pc1S', 'pc2S', 'pc3S', 'pc1P', 'pc2P', 'pc3P', 'pP'
            ]
casesSecondaires= [
       'ai1S', 'ai2S', 'ai3S', 'ai1P', 'ai2P', 'ai3P', 'is1S', 'is2S', 'is3S', 'is1P', 'is2P', 'is3P'
            ]
casesTotales=casesPrincipales+casesSecondaires

###Création de la colonne de fréquence *freq*
- fréquence positive : 10.f
- fréquence nulle : 5
- fréquence inexistante : 1

In [46]:
bdlexique["freq"]=bdlexique[bdlexique["formeFrantext"]!="***"]["formeFrantext"].astype(float)*10
bdlexique.loc[bdlexique["formeFrantext"]=="0","freq"]=5
bdlexique.loc[bdlexique["formeFrantext"]=="***","freq"]=1

###Elimination des colonnes inutiles
- indicateur L23
- fréquences brutes

In [47]:
inutilesColonnes=["L23","lemmeFrantext","lemmeFilms","formeFrantext","formeFilms"]
for colonne in inutilesColonnes:
    del bdlexique[colonne]

###Extraction des formes verbales dans une structure *verbes*

In [48]:
verbes=bdlexique[bdlexique["cs"]=="V"].copy()

In [49]:
verbes["phono"]=verbes["phono"].apply(lambda x: recoder(x))

In [50]:
#verbes["phono"]
print recoder(u"tOtòtètE")

tOtOtEtE


###Ajout d'une colonne *prob* pour la fréquence relative

In [51]:
verbes["prob"]=verbes["freq"]/verbes["freq"].sum()

###Ajout d'une colonne *case*
- remplir les *ms* vide pour permettre la concaténation sans erreur
- *case* est la concaténation de *vs* et *ms*

In [52]:
verbes["ms"]=verbes["ms"].fillna("")
verbes["case"]=verbes["vs"]+verbes["ms"]

#Echantillonage

##Tirage incrémental des formes disponibles
- increment : taille de chaque tirage
- nombre : nombre de tirages

np.random.choice donne une liste d'index de formes tirées dans l'ordre du tirage

on découpe la liste en morceaux de la taille de l'incrément

In [53]:
nombre=1000
increment=250
tirage=np.random.choice(verbes.index,size=nombre*increment,p=verbes["prob"],replace=False)
tirages=[tirage[increment*x:increment*(x+1)] for x in range(len(tirage)/increment+(len(tirage)%increment!=0))]

##Boucle sur les échantillons successifs
- extrait contient les index des formes du lot
- paradigmes contient la table des formes du lot

In [54]:
'''
lot=[]
tailles={}
for n in range(4):
    lot.extend(tirages[n])
    extrait=verbes.ix[np.sort(lot)]
    paradigmes=pd.pivot_table(extrait, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: x)
    if n%25==0:
        tailles[n]=paradigmes.count()
'''

"\nlot=[]\ntailles={}\nfor n in range(4):\n    lot.extend(tirages[n])\n    extrait=verbes.ix[np.sort(lot)]\n    paradigmes=pd.pivot_table(extrait, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: x)\n    if n%25==0:\n        tailles[n]=paradigmes.count()\n"

##Assembler les échantillons correspondant à une étape n

In [55]:
def assemblerExtrait(nombre):
    return verbes.ix[np.sort(np.concatenate(tirages[0:nombre]))]

In [56]:
extrait=assemblerExtrait(20)
paradigmes=pd.pivot_table(extrait, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()

In [57]:
def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
#    return (same,diff1,diff2,diff1+"_"+diff2)
    return (diff1+"-"+diff2)

def rowDiff(row, patrons):
    result=diff(row[0],row[1])
    if not result in patrons:
        patrons[result]=(formesPatron(),formesPatron())
    patrons[result][0].ajouterFormes(row[0])
    patrons[result][1].ajouterFormes(row[1])
    return (result[0],result[1])


In [58]:
def patron2regexp(morceaux):
    result="^"
    for morceau in morceaux:
        if morceau=="*":
            result+="(.*)"
        elif len(morceau)>1:
            result+="(["+morceau+"])"
        else:
            result+=morceau
    result+="$"
    result=result.replace(")(","")
    return result

In [59]:
class formesPatron:
    '''
    Accumulateur de formes correspondant à un patron pour calcul de la Généralisation Minimale (cf. MGL)
    '''
    def __init__(self):
        self.formes=[]

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterForme(self,forme):
        self.formes.append(forme)
        
    def calculerGM(self):
        minLongueur=len(min(self.formes, key=len))
        maxLongueur=len(max(self.formes, key=len))
        if debug: print minLongueur, maxLongueur
        positions=[]
        if maxLongueur>minLongueur:
            positions.append("*")
        for i in xrange(minLongueur, 0, -1):
            phonemes=set([x[-i] for x in self.formes])
            if debug: print phonemes
            if "." in phonemes:
                positions.append(".")
            else:
                positions.append("".join(fs.lattice[phonemes].extent))
        return patron2regexp(positions)

class pairePatrons:
    '''
    Accumulateur de triplets (f1,f2,patron) correspondant à une paire pour calcul des Généralisations Minimales (cf. MGL)
    '''
    def __init__(self,nom=""):
        self.patrons1={}
        self.patrons2={}
        self.nom=nom

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterFormes(self,forme1,forme2,patron):
        patron12=patron
        (pat1,pat2)=patron.split("-")
        patron21=pat2+"-"+pat1
        if not patron12 in self.patrons1:
            self.patrons1[patron12]=formesPatron()
        self.patrons1[patron12].ajouterForme(forme1)
        if not patron21 in self.patrons2:
            self.patrons2[patron21]=formesPatron()
        self.patrons2[patron21].ajouterForme(forme2)
        
        
    def calculerGM(self):
        resultat1={}
        for patron in self.patrons1:
            if debug: print "patron1", patron
            resultat1[patron]=self.patrons1[patron].calculerGM()
        resultat2={}
        for patron in self.patrons2:
            if debug: print "patron2", patron
            resultat2[patron]=self.patrons2[patron].calculerGM()
        return (resultat1,resultat2) 

##Appliquer la formule de calcul des différences entre chaines à chaque ligne

>si il y a au moins une ligne

>>on applique la différence à la ligne

>>on calcule les deux patrons par suppression des points initiaux

>>on renvoie le groupement par patrons (1&2)

>sinon

>>on renvoie le paradigme vide d'origine

In [64]:
def rapports(paradigme):
    patrons=pairePatrons()
    if len(paradigme)>0:
        for index, row in paradigme.iterrows():
            patrons.ajouterFormes(row[0],row[1],diff(row[0],row[1]))
            print row[1]
        (regles1,regles2)=patrons.calculerGM()
    return patrons.calculerGM()

###Dédoubler les lignes avec des surabondances dans *colonne*
>identifier une ligne avec surabondance

>>ajouter les lignes correspondant à chaque valeur

>>ajouter le numéro de la ligne initiale dans les lignes à supprimer

>supprimer les lignes avec surabondance

NB : il faut préparer le tableau pour avoir une indexation qui permette l'ajout des valeurs individuelles et la suppression des lignes de surabondances

In [61]:
def splitCellMates(df,colonne):
    '''
    Calcul d'une dataframe sans surabondance par dédoublement des valeurs
    '''
    test=df.reset_index()
    del test["index"]
    splitIndexes=[]
    for index,ligne in test.iterrows():
        if "," in ligne[colonne]:
            valeurs=set(ligne[colonne].split(","))
            nouvelleLigne=ligne
            for valeur in valeurs:
                nouvelleLigne[colonne]=valeur
                test=test.append(nouvelleLigne,ignore_index=True)
            splitIndexes.append(index)
    if splitIndexes:
        test=test.drop(test.index[splitIndexes])
    return test


##Calculer les rapports entre formes pour chaque paire

>on fait la liste des cases de *paradigmes*

>pour chaque paire du tableau principal

>>si la paire fait partie des cases de *paradigmes*

>>>on calcule le rapport

>>sinon

>>>on signale que qu'une des cases n'est pas représentée

In [62]:
def evaluerEchantillon(paradigmes):
    result={}
    colonnes=paradigmes.columns.values.tolist()
    for paire in it.combinations(casesPrincipales,2):
        paireListe=list(paire)
        print paireListe,
        paireListe.append("lexeme")
        if paire[0] in colonnes and paire[1] in colonnes:
            paradigmePaire=paradigmes[paireListe].dropna(thresh=3, axis=0).reindex()
            paradigmePaire=splitCellMates(splitCellMates(paradigmePaire,paire[0]),paire[1])
            result[paire]=rapports(paradigmePaire)
        else:
            result[paire]=("missing pair", paire)
    return result

###Boucle de calcul des analogies pour l'échantillon

In [65]:
%%time
debug=False
resultats=evaluerEchantillon(paradigmes)

['inf', 'pi1S'] abâdOn
aksEpt
akôpaJ
ak9j
aksElEr
aSEt
adOr
Ed
Em
vE
alôZ
amEn
amyz
amEljOr
anôs
apEl
aplik
apOrt
aprâ
aprEsi
apHi
araS
arâZ
ariv
arEt
aspir
asyr
ataS
atak
atâ
atEri
atir
atrap
Ogmât
aval
E
bEz
ba
blEs
bwa
bOs
buZ
bryl
kaS
kalm
kas
koz
sEs
SâZ
Sât
SarZ
SErS
Swazi
kOJ
kOmâd
kOmâs
kôprâ
kôt
kôdHi
kôfirm
kôfô
kOnE
kôsidEr
kôstat
kôtiny
kuS
kup
kuvr
kut
krak
krEv
kri
krwa
d6mâd
dEsâ
d6vjê
d6vin
dwa
di
diriZ
diskyt
dOn
dOr
dut
dEbaras
dEsid
dEkOn
dEkuvr
dEfE
dEfâ
dEZ9n
dElivr
dEpâs
dEpoz
dErâZ
dEzabij
âbras
âmEn
âmEnaZ
âpOrt
âprût
âpES
âfErm
âfôs
âfHi
âlEv
âtâ
âtEr
âtr
âvi
âvwa
EspEr
EsEj
EsHi
Ekskyz
Egzist
Esplik
Esploz
fE
fErm
fil
fôs
fu
frEn
fym
fElisit
gaJ
gard
gar
grat
grêp
gaS
abij
imaZin
êkjEt
êstal
ZEt
Zwê
Zu
Zyr
lEs
lâs
lav
lEv
libEr
li
lu
laS
mâd
mâZ
mâk
marS
mari
mark
m6nas
mâ
mE
môt
môtr
mOk
m9r
mEl
nEgOsi
ObEi
Okyp
Ofr
ubli
uvr
pardOn
parl
partaZ
par
pas
pEj
pâs
pEr
pik
plas
plê
plE
plan
plâk
pl9r
pOrt
poz
pOsEd
pus
p2
prâ
pri
prOmE
prOtEZ
pruv
prOvOk
prEpar
prE

KeyboardInterrupt: 

In [29]:
resultats[('pi2S', 'pi3S')][1]

{'-': u'^(.*[ptkbdgfsSvzZmnJjlrwHiyEe926auOo\xea\xfb\xe2\xf4])$',
 u'je-wa': u'^asje$',
 u'wa-je': u'^aswa$'}

In [25]:
colonnes=paradigmes.columns.values.tolist()
result={}
paire=('inf','fi1S')
paireListe=list(paire)
print paireListe
paireListe.append("lexeme")
if paire[0] in colonnes and paire[1] in colonnes:
    paradigmePaire=paradigmes[paireListe].dropna(thresh=3, axis=0).reindex()
    paradigmePaire=splitCellMates(splitCellMates(paradigmePaire,paire[0]),paire[1])
    result[paire]=rapports(paradigmePaire)
else:
    result[paire]=("missing pair", paire)

['inf', 'fi1S']
patron1 -E
3 6
set([u'r', u'\xe2', u'd'])
set([u'i', u'd'])
set([u'r'])
patron1 vwar-rE
6 6
set([u'p'])
set([u'u'])
set([u'v'])
set([u'w'])
set([u'a'])
set([u'r'])
patron1 6ni.-jêd.E
5 5
set([u'v'])
set([u'6'])
set([u'n'])
set([u'i'])
set([u'r'])
patron1 E.-6.E
3 3
set([u'f'])
set([u'E'])
set([u'r'])
patron1 ale-irE
3 3
set([u'a'])
set([u'l'])
set([u'e'])
patron1 Et.-s6.E
3 3
set([u'E'])
set([u't'])
set([u'r'])
patron2 E-
4 7
set([u'r', u'\xe2', u'd'])
set([u'i', u'd'])
set([u'r'])
set([u'E'])
patron2 irE-ale
3 3
set([u'i'])
set([u'r'])
set([u'E'])
patron2 6.E-E.
4 4
set([u'f'])
set([u'6'])
set([u'r'])
set([u'E'])
patron2 jêd.E-6ni.
6 6
set([u'v'])
set([u'j'])
set([u'\xea'])
set([u'd'])
set([u'r'])
set([u'E'])
patron2 s6.E-Et.
4 4
set([u's'])
set([u'6'])
set([u'r'])
set([u'E'])
patron2 rE-vwar
4 4
set([u'p'])
set([u'u'])
set([u'r'])
set([u'E'])


âêûô


In [ ]:
echantillonVerbes=["chanter","manger","asseoir","avoir","prendre","dormir"]
extrait=verbes[verbes["lexeme"].isin(echantillonVerbes)]

###Initialisation pour l'échantillon complet

In [26]:
extrait=verbes

In [81]:
paradigmes

case,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,pi2P,pi2S,pi3P,pi3S,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
0,abaisser,NaN,abEsE,NaN,NaN,NaN,NaN,NaN,abEs6rE,NaN,...,abEse,abEs,abEs,abEs,NaN,abEs,NaN,abEs,abEs,abEs
1,abandonner,NaN,abâdOnE,NaN,NaN,abâdOnEr,abâdOna,abâdOn6rô,abâdOn6rE,abâdOn6re,...,abâdOne,abâdOn,abâdOn,abâdOn,abâdOnjô,abâdOn,abâdOnje,abâdOn,abâdOn,abâdOn
2,abasourdir,NaN,NaN,NaN,abazurdi,NaN,NaN,NaN,NaN,NaN,...,NaN,abazurdi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,abattre,NaN,NaN,NaN,NaN,abatir,abati,NaN,abatrE,abatre,...,abate,aba,abat,aba,NaN,abat,abatje,abat,abat,abat
4,abcéder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,abdiquer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,abdik,abdik,NaN,abdik,NaN,NaN,NaN,abdik
6,aberrer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,abEr,NaN,NaN,NaN,NaN,abEr,NaN
7,abhorrer,NaN,NaN,abOrat,NaN,NaN,NaN,NaN,NaN,NaN,...,abOre,NaN,NaN,abOr,NaN,abOr,NaN,NaN,NaN,abOr
8,abjurer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,abZyr6re,...,abZyre,NaN,NaN,abZyr,NaN,abZyr,NaN,NaN,NaN,abZyr
9,ablater,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
paradigmes[paradigmes["lexeme"]=="asseoir"]

case,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,pi2P,pi2S,pi3P,pi3S,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
332,asseoir,NaN,asi,NaN,asi,asir,asi,NaN,"asjErE,aswarE",NaN,...,"asEje,aswaje","asje,aswa","asEj,aswa","asje,aswa",asEjô,"asEj,aswa",NaN,"asEj,aswa","asEj,aswa","asEj,aswa"


In [103]:
paire=("pi1S","pI2S")
paireListe=list(paire)
paireListe.append("lexeme")
paradigmePaire=paradigmes[paireListe].dropna(thresh=3, axis=0).reindex()
paradigmePaire=splitCellMates(splitCellMates(paradigmePaire,paire[0]),paire[1])
paradigmePaire["rapports"]=paradigmePaire.apply(rowDiff, axis=1)
paradigmePaire["patron1"]=paradigmePaire["rapports"].apply(lambda x: x[0].lstrip("."))
paradigmePaire["patron2"]=paradigmePaire["rapports"].apply(lambda x: x[1].lstrip("."))
print paradigmePaire
paradigmePaire.groupby(["patron1","patron2"])[["lexeme","patron1","patron2"]].apply(lambda x: ", ".join(x["lexeme"]))

case      pi1S      pI2S       lexeme              rapports patron1 patron2
0         abEs      abEs     abaisser          (...., ....)                
1       abâdòn    abâdòn   abandonner      (......, ......)                
2     abazurdi  abazurdi   abasourdir  (........, ........)                
3          aba       aba      abattre            (..., ...)                
4        absEd     absEd      abcéder        (....., .....)                
5        abdik     abdik     abdiquer        (....., .....)                
6         abEk      abEk     abecquer          (...., ....)                
7         abEr      abEr      aberrer          (...., ....)                
8         abOr      abOr     abhorrer          (...., ....)                
9        abZyr     abZyr      abjurer        (....., .....)                
10       ablat     ablat      ablater        (....., .....)                
11      ablati    ablati      ablatir      (......, ......)                
12       abl

patron1  patron2
                    abaisser, abandonner, abasourdir, abattre, abc...
2        9j                                                   vouloir
Ej       è                                                 sous-payer
Hi       wa                                                      être
je       wa                                                   asseoir
wa       je                                                   asseoir
âvè      vatâ                                                en aller
è        Ej                                                sous-payer
         a                                                      aller
         aS                                                    savoir
dtype: object

In [72]:
paire=("pi1S","pI2S")
paireListe=list(paire)
paireListe.append("lexeme")
paradigmePaire=paradigmes[paireListe].dropna(thresh=3, axis=0).reset_index().reindex()
paradigmePaire

case,index,pi1S,pI2S,lexeme
0,0,abâdòn,abâdòn,abandonner
1,11,adOr,adOr,adorer
2,17,Em,Em,aimer
3,18,vè,va,aller
4,26,apEl,apEl,appeler
5,31,aprèsi,aprèsi,apprécier
6,34,ariv,ariv,arriver
7,36,arEt,arEt,arrêter
8,42,atâ,atâ,attendre
9,45,avâs,avâs,avancer


In [74]:
paradigmePaire=splitCellMates(paradigmePaire,paire[0])
paradigmePaire

case,index,pi1S,pI2S,lexeme
0,0,abâdòn,abâdòn,abandonner
1,11,adOr,adOr,adorer
2,17,Em,Em,aimer
3,18,vè,va,aller
4,26,apEl,apEl,appeler
5,31,aprèsi,aprèsi,apprécier
6,34,ariv,ariv,arriver
7,36,arEt,arEt,arrêter
8,42,atâ,atâ,attendre
9,45,avâs,avâs,avancer


In [43]:
for name, group in paradigmePaire.groupby(["patron1","patron2"]):#[["lexeme","patron1","patron2"]].apply(lambda x: ", ".join(x["lexeme"])):
    print name
    for element in group:
        print group[element]

case pi1S  fi1P   lexeme    rapports patron1 patron2
47      è   òrô    avoir    (è, òrô)       è     òrô
332    p2  purô  pouvoir  (.2, .urô)       2     urô
(u'2', u'ur\xf4')
332    p2
Name: pi1S, dtype: object
332    purô
Name: fi1P, dtype: object
332    pouvoir
Name: lexeme, dtype: object
332    (.2, .urô)
Name: rapports, dtype: object
332    2
Name: patron1, dtype: object
332    urô
Name: patron2, dtype: object
(u'\xe8', u'\xf2r\xf4')
47    è
Name: pi1S, dtype: object
47    òrô
Name: fi1P, dtype: object
47    avoir
Name: lexeme, dtype: object
47    (è, òrô)
Name: rapports, dtype: object
47    è
Name: patron1, dtype: object
47    òrô
Name: patron2, dtype: object


In [ ]:
test=paradigmes[["lexeme","pi1S","pi3S"]].dropna(thresh=2, axis=0).reindex()
test

In [ ]:
test=splitCellMates(test,"pi1S")
splitCellMates(test,"pi3S")

##Tirages pour les statistiques de remplissage des paradigmes

In [ ]:
manquantesMoyenne={}
annexesMoyenne={}
%%time
nbTirages=250
echantillonTailles=[250*x for x in range(2,10)]

def remplissageParadigmes(nbTirages,echantillonTailles)
    for echantillonTaille in echantillonTailles:
        nbManquantes=0
        nbAnnexes=0
        for n in range(nbTirages):
            tirage=np.random.choice(verbes.index,size=echantillonTaille,p=verbes["prob"],replace=False)
            extrait=verbes.ix[np.sort(tirage)]
            paradigmes=pd.pivot_table(extrait, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: x)
            paradigmes.count()
            manquantes=[case for case in casesPrincipales if not case in extrait[extrait["lexeme"].isin(casesPrincipales)]["case"].unique()]
            annexes=[case for case in casesSecondaires if not case in extrait[extrait["lexeme"].isin(casesSecondaires)]["case"].unique()]
            if manquantes :
                nbManquantes+=1
            if annexes:
                nbAnnexes+=1
        manquantesMoyenne[echantillonTaille]=100*float(nbManquantes)/nbTirages
        annexesMoyenne[echantillonTaille]=100*float(nbAnnexes)/nbTirages

In [ ]:
paradigmes=pd.pivot_table(extrait, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: x)
paradigmes

In [ ]:
tirage.dtype

In [ ]:
tirages=[]
increment=250
resteVerbes=verbes
resteVerbes.index
for i in range(32):
    tirage=np.random.choice(resteVerbes.index,size=increment,p=resteVerbes["prob"],replace=False)
    print tirage
    resteVerbes.drop(resteVerbes.index[tirage])
    print resteVerbes.count()
    tirages.append(tirage)

In [ ]:
formes=[]
for tirage in tirages:
    formes.extend(tirage)
extrait=verbes.ix[np.sort(formes)]

In [ ]:
paradigmes=pd.pivot_table(extrait, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: x)

In [ ]:
resteVerbes.index

In [ ]:
%%time
nbTirages=250
echantillonTailles=[250*x for x in range(2,10)]

for echantillonTaille in echantillonTailles:
    nbManquantes=0
    nbAnnexes=0
    for n in range(nbTirages):
        tirage=np.random.choice(verbes.index,size=echantillonTaille,p=verbes["prob"],replace=False)
        extrait=verbes.ix[np.sort(tirage)]
        formes=extrait.groupby(["lexeme"])["case"].count()
        liste=formes[formes>1].index
        manquantes=[case for case in casesPrincipales if not case in extrait[extrait["lexeme"].isin(liste)]["case"].unique()]
        annexes=[case for case in casesSecondaires if not case in extrait[extrait["lexeme"].isin(liste)]["case"].unique()]
        if manquantes :
            nbManquantes+=1
        if annexes:
            nbAnnexes+=1
    manquantesMoyenne[echantillonTaille]=100*float(nbManquantes)/nbTirages
    annexesMoyenne[echantillonTaille]=100*float(nbAnnexes)/nbTirages

In [ ]:
courbeSecondaires=pd.Series(annexesMoyenne)
courbePrincipales=pd.Series(manquantesMoyenne)

In [ ]:
courbePrincipales.plot()
courbeSecondaires.plot()

In [ ]:
formes[formes>1]

In [ ]:
tirageCases=extrait[extrait["lexeme"].isin(liste)].groupby(["lexeme"])["case"].apply(lambda x: list(x)).copy()
#tirageCases["paires"]=tirageCases["cases"].apply(lambda x: [(y,z) for (y,z) in it.product(x,x) if y!=z])

In [ ]:
nbPaires={}
for paire in produitPrincipal:
#    print paire
    for index,row in tirageCases.iteritems():
#        print row
        if paire[0] in row and paire[1] in row:
            if not paire in nbPaires:
                nbPaires[paire]=0
            nbPaires[paire]+=1

In [ ]:
nbPairesSuffisantes=pd.Series()
for seuil in range(10):
    pairesManquantes=produitPrincipal[:]
    for paire in nbPaires:
        if nbPaires[paire]>seuil:
            pairesManquantes.remove(paire)
    nbPairesSuffisantes.loc[seuil]=len(produitPrincipal)-len(pairesManquantes)